<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/nafnet_video_continue_new_buffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Перейити в гугл диск и загрузить в корень диска файл 

https://drive.google.com/drive/ 



In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла: ") or "input.mp4"

In [ ]:
#@title ##**Проверка GPU** { display-mode: "form" }
!nvidia-smi

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
%cd /content

!git clone https://github.com/megvii-research/NAFNet

In [ ]:
#@title ##**Установить библиотеку** { display-mode: "form" }
%cd /content/NAFNet

!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

In [ ]:
#@title ##**Импорт библиотеки** { display-mode: "form" }
%cd /content/NAFNet

import gdown

import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from google.colab import files
import shutil
import glob

In [ ]:
#@title ##**Скачать модель** { display-mode: "form" }
%cd /content/NAFNet

# Deblur
if not os.path.exists("./experiments/pretrained_models/NAFNet-REDS-width64.pth"):
  gdown.download('https://drive.google.com/uc?id=14D4V4raNYIOhETfcuuLI3bGLB-OYIv6X', "./experiments/pretrained_models/", quiet=False)

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Создать папки для изображений** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт

%cd /content/NAFNet

deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'

if not os.path.exists(deblur_upload_folder):
    os.makedirs(deblur_upload_folder)


if not os.path.exists(deblur_result_folder):
    os.makedirs(deblur_result_folder)


In [ ]:
#@title ##**Очистить входящую папку** { display-mode: "form" }
if os.path.isdir(deblur_upload_folder):
    shutil.rmtree(deblur_upload_folder)

os.makedirs(deblur_upload_folder)

In [ ]:
#@title ##**Очистить исходящую папку** { display-mode: "form" }
if os.path.isdir(deblur_result_folder):
    shutil.rmtree(deblur_result_folder)

os.makedirs(deblur_result_folder)

In [ ]:
#@title ##**Секвенция видео** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт
import cv2
import imageio
import os
import tqdm
import subprocess
import numpy as np
import time

deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'

library = "imageio" #@param ["cv2","pyav","imageio","ffmpeg","skvideo","scipy","moviepy"]
delay = "0.05" #@param [0, 0.01, 0.05, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
if (library == "cv2"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    cmd = 'ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

    cmd = 'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()
    vidcap = cv2.VideoCapture(full_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    vidcap = cv2.VideoCapture(full_path)
    success, image = vidcap.read()
    
    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_cv2 = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    while success:
        try:
            cv2.imwrite(f"/mnt/gdrive/MyDrive/deblur_input/frame{int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):09d}.jpg", image)
        except cv2.error:
            print("Error writing to disk. Retrying...")
            continue
        pbar_cv2.update(1)
        time.sleep(float(delay))
        success, image = vidcap.read()

    pbar_cv2.close()
    vidcap.release()

elif (library == "pyav"):
    !pip install av
    import av
    video = av.open(full_path)

    fps_value = video.streams.video[0].average_rate
    frame_count = video.streams.video[0].frames

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)
        print("Duration: ", duration)
        print("Frames: ", frame_count)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_pyav = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for packet in video.demux():
            for frame in packet.decode():
                img = frame.to_ndarray(format='rgb24')
                img = img.astype('uint8')  # преобразование к типу uint8
                img_path = f"/mnt/gdrive/MyDrive/deblur_input/frame{packet.pts:09d}.jpg"
                imageio.imwrite(img_path, img)
                pbar_pyav.update(1)
                time.sleep(float(delay))

        pbar_pyav.close()
        video.close()
    else:
        print("Error: FPS value is None.")


elif (library == "imageio"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = imageio.get_reader(full_path)

    fps = video.get_meta_data()['fps']
    frame_count = video.count_frames()
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_imageio = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
      img_path = f"/mnt/gdrive/MyDrive/deblur_input/frame{i+1:09d}.jpg"
      if os.path.isfile(img_path):
          pbar_imageio.update(1)
          continue
      while True:
          try:
              imageio.imwrite(img_path, frame)
          except Exception as e:
              print(f"Error writing to disk: {str(e)}. Retrying...")
              continue
          break
      pbar_imageio.update(1)
      time.sleep(float(delay))

    pbar_imageio.close()
    video.close()

elif (library == "ffmpeg"):
    !pip install ffmpeg-python
    import ffmpeg
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    probe = ffmpeg.probe(full_path)
    video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    fps = video_info['r_frame_rate']
    duration = float(video_info['duration'])
    frame_count = int(video_info['nb_frames'])

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    pbar_ffmpeg = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    process = (
        ffmpeg
        .input(full_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', qscale=0)
        .run_async(pipe_stdout=True)
    )

    for i in range(frame_count):
        try:
            raw_video = process.stdout.read(video_info['width'] * video_info['height'] * 3)
            frame = np.frombuffer(raw_video, dtype='uint8').reshape((video_info['height'], video_info['width'], 3))
            frame_path = f"/mnt/gdrive/MyDrive/deblur_input/frame{i:09d}.jpg"
            if os.path.isfile(frame_path):
              pbar_ffmpeg.update(1)
              continue
            imageio.imwrite(frame_path, frame)
        except Exception as e:
            print(f"Error writing to disk: {str(e)}. Retrying...")
            continue
        pbar_ffmpeg.update(1)
        time.sleep(float(delay))

    pbar_ffmpeg.close()
    process.wait()
elif (library == "skvideo"):
    !pip install scikit-video
    import skvideo.io
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = skvideo.io.vread(full_path)

    fps_str = skvideo.io.ffprobe(full_path)['video']['@avg_frame_rate']
    fps_parts = fps_str.split('/')
    fps = int(fps_parts[0]) / int(fps_parts[1])
    frame_count = len(video)
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    outputdict = {'-q:1': '1'}
    pbar_skvideo = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
        while True:
            try:
                skvideo.io.vwrite(f"/mnt/gdrive/MyDrive/deblur_input/frame{i:09d}.jpg", frame, outputdict=outputdict)
            except Exception as e:
                print(f"Error writing to disk: {str(e)}. Retrying...")
                continue
            break
        pbar_skvideo.update(1)
        time.sleep(float(delay))

    pbar_skvideo.close()

elif (library == "scipy"):
    import scipy.misc
    import scipy.ndimage
    
    full_path = os.path.join(path, file_name)
    video = imageio.get_reader(full_path)
    fps_value = video.get_meta_data()['fps']
    frame_count = video.get_length()

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_scipy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for i, im in enumerate(video):
            img_path = f"/mnt/gdrive/MyDrive/deblur_input/frame{i+1:09d}.jpg"
            im = Image.fromarray(im)
            im.save(img_path, quality=100)
            pbar_scipy.update(1)
            time.sleep(float(delay))

        pbar_scipy.close()
    else:
        print("Error: FPS value is None.")

elif library == "moviepy":
    from moviepy.editor import VideoFileClip
    from PIL import Image
    full_path = os.path.join(path, file_name)
    video = VideoFileClip(full_path)
    fps_value = video.fps
    frame_count = int(video.duration * fps_value)

    print("FPS: ", fps_value)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_moviepy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video.iter_frames()):
        img_path = f"/mnt/gdrive/MyDrive/deblur_input/frame{i+1:09d}.jpg"
        im = Image.fromarray(frame)
        im.save(img_path, quality=100)
        pbar_moviepy.update(1)
        time.sleep(float(delay))

    pbar_moviepy.close()

In [ ]:
#@title ##**Проверка на попущенные кадры (опционально)** { display-mode: "form" }
import os

frame_dir = '/mnt/gdrive/MyDrive/deblur_input'
frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
min_frame = min(frames)
max_frame = max(frames)
print(min_frame)
print(max_frame)

missing_frames = []
for i in range(min_frame, max_frame+1):
    if i not in frames:
        missing_frames.append(i)

if len(missing_frames) > 0:
    print(f"Missing frames: {missing_frames}")
else:
    print("All frames present")
    

In [ ]:
#@title ##**Определение функции** { display-mode: "form" }
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

def sr_display(LR_l, LR_r, SR_l, SR_r):
  h,w = SR_l.shape[:2]
  LR_l = cv2.resize(LR_l, (w,h), interpolation=cv2.INTER_CUBIC)
  LR_r = cv2.resize(LR_r, (w,h), interpolation=cv2.INTER_CUBIC)
  fig = plt.figure(figsize=(w//40, h//40))
  ax1 = fig.add_subplot(2, 2, 1) 
  plt.title('Input image (Left)', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(2, 2, 2)
  plt.title('NAFSSR output (Left)', fontsize=16)
  ax2.axis('off')
  ax1.imshow(LR_l)
  ax2.imshow(SR_l)

  ax3 = fig.add_subplot(2, 2, 3) 
  plt.title('Input image (Right)', fontsize=16)
  ax3.axis('off')
  ax4 = fig.add_subplot(2, 2, 4)
  plt.title('NAFSSR output (Right)', fontsize=16)
  ax4.axis('off')
  ax3.imshow(LR_r)
  ax4.imshow(SR_r)

  plt.subplots_adjust(wspace=0.04, hspace=0.04)

def stereo_image_inference(model, img_l, img_r, save_path):
      img = torch.cat([img_l, img_r], dim=0)
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      img_L = visuals['result'][:,:3]
      img_R = visuals['result'][:,3:]
      img_L, img_R = tensor2img([img_L, img_R])
      
      imwrite(img_L, save_path.format('L'))
      imwrite(img_R, save_path.format('R'))


In [ ]:
#@title ##**Построить модель** { display-mode: "form" }
%cd /content/NAFNet

opt_path = 'options/test/REDS/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

In [ ]:
#@title ##**Deblur изображений** { display-mode: "form" }
%cd /content/NAFNet
import glob
import os
import time
from tqdm import tqdm

deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'

upload_files = set()
while not upload_files:
    try:
        upload_files = set(os.listdir(deblur_upload_folder))
    except OSError as e:
        print(f"Error: {e}, retrying...")
        time.sleep(2)

result_files = set(os.listdir(deblur_result_folder))

input_list = [file_path for file_path in glob.glob(os.path.join(deblur_upload_folder, '*'))
              if os.path.basename(file_path) not in result_files and os.path.basename(file_path) in upload_files]


for input_path in tqdm(input_list, desc="Processing images"):
    img_input = imread(input_path)
    inp = img2tensor(img_input)
    output_path = os.path.join(deblur_result_folder, os.path.basename(input_path))
    single_image_inference(NAFNet, inp, output_path)



In [ ]:
#@title ##**Проверка на попущенные кадры (опционально)** { display-mode: "form" }
import os

frame_dir_output = '/mnt/gdrive/MyDrive/deblur_output'
frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir_output) if f.endswith('.jpg')]
min_frame = min(frames)
max_frame = max(frames)
print(min_frame)
print(max_frame)

missing_frames = []
for i in range(min_frame, max_frame+1):
    if i not in frames:
        missing_frames.append(i)

if len(missing_frames) > 0:
    print(f"Missing frames: {missing_frames}")
else:
    print("All frames present")

In [ ]:
#@title ##**Наложение входящего и исходящего** { display-mode: "form" }
import os
from PIL import Image
import time

opacity = "100" #@param [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
if opacity != 100:
  # Пути к входящей и исходящей папкам
  deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
  deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'
  deblur_result_folder_overlay = '/mnt/gdrive/MyDrive/deblur_output_overlay'

  if os.path.isdir(deblur_result_folder_overlay):
      shutil.rmtree(deblur_result_folder_overlay)
  os.makedirs(deblur_result_folder_overlay)

  # Получение списка имен файлов в папке
  max_retries = 10
  retries = 0
  while retries < max_retries:
      try:
          filenames = os.listdir(deblur_upload_folder)
          break
      except:
          print("Ошибка получения списка файлов. Попытка", retries+1, "из", max_retries)
          retries += 1
          time.sleep(2)

  for filename in filenames:
      # Открытие изображений
      try:
          img1 = Image.open(os.path.join(deblur_upload_folder, filename))
          img2 = Image.open(os.path.join(deblur_result_folder, filename))
      except:
          print(f"Ошибка открытия файла  {filename}")
          time.sleep(2)
          continue

      
      img1 = img1.convert("RGBA")
      img2 = img2.convert("RGBA")

      # Получение размеров изображений
      width1, height1 = img1.size
      width2, height2 = img2.size

      # Изменение прозрачности второго изображения
      img2.putalpha(int(float(opacity) * 2.55))

      # Сравнение размеров и изменение большего изображения
      if width1 * height1 > width2 * height2:
          img2 = img2.resize((width1, height1))
      else:
          img1 = img1.resize((width2, height2))

      # Наложение изображений с прозрачностью 50%
      result = Image.alpha_composite(img1, img2)

      # Преобразование режима изображения в RGB
      result = result.convert("RGB")

      # Сохранение полученного изображения
      result.save(os.path.join(deblur_result_folder_overlay, filename.replace(".jpg", ".png")), quality=100)
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output_overlay'

In [ ]:
#@title ##**Сборка видео** { display-mode: "form" }
import cv2
import os
from tqdm.notebook import tqdm

output_file = "/mnt/gdrive/MyDrive/output.mp4"
path = '/mnt/gdrive/MyDrive/'

full_path = os.path.join(path, file_name)

cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

img_files = os.listdir(deblur_result_folder)
img_files.sort()

frame = cv2.imread(os.path.join(deblur_result_folder, img_files[0]))
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, fps_of_video, (width, height))

for img_file in tqdm(img_files, desc='Сборка видео'):
    img_path = os.path.join(deblur_result_folder, img_file)
    frame = cv2.imread(img_path)

    writer.write(frame)

writer.release()
print('Сборка завершена')

In [ ]:
#@title ##**Добавление звука** { display-mode: "form" }
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip

video = VideoFileClip(output_file)

audio = AudioFileClip(full_path)

video = video.set_audio(audio)

video.write_videofile('/mnt/gdrive/MyDrive/output_with_audio.mp4')


In [ ]:
#@title ##**Скачать файл (опционально, быстерее можно скачать в гугл диске)** { display-mode: "form" }
from google.colab import files
files.download('/mnt/gdrive/MyDrive/output_with_audio.mp4')